In [4]:
import os
import pandas as pd
import numpy as np

In [ ]:
#폴더, 파일명, 간격 범위 기입
folder = r"c:\Users\limye\OneDrive - gachon.ac.kr\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\2. Collecting data\250624_12개의 샘플\minimized_data\depth"

file_name_str = "D3 D4 D5 D6" #"" 안에 띄어쓰기로 구별

gap_value = 0.2 #0.2 = 20 % 만큼 행 띄우기

In [6]:
def merge_data(file_name_str, folder, output_path):
    file_names = file_name_str.split()
    file_paths = [os.path.join(folder, f"{name}.csv") for name in file_names]

    dataframes = [pd.read_csv(fp, header=None) for fp in file_paths]
    n_rows = len(dataframes[0])

    last_time = dataframes[0].iloc[-1, 0]
    rounded_time = round(last_time)
    time_gap = rounded_time / n_rows
    print(f"자동 계산된 time_gap: {time_gap:.6f}")

    gap = int(gap_value * n_rows)

    total_rows = n_rows + (len(dataframes) - 1) * (n_rows + gap)

    if total_rows >= 1_000_000:
        print(f"⚠ 경고: 생성될 파일의 행 수가 {total_rows:,}개로 1,000,000을 초과합니다. 파일 크기에 주의하세요.")

    n_cols = len(dataframes) + 1
    result_array = np.full((total_rows, n_cols), np.nan)

    result_array[:, 0] = np.arange(0, total_rows) * time_gap

    row_pos = 0
    for i, df in enumerate(dataframes):
        result_array[row_pos:row_pos + n_rows, i + 1] = df.iloc[:, 1].values
        row_pos += n_rows + gap  # 항상 동일 gap 추가

    pd.DataFrame(result_array).to_csv(output_path, index=False, header=False, float_format="%.6f")
    print(f"병합된 파일이 저장되었습니다: {output_path}")
    
output_path = os.path.join(folder, "00Merge.csv")

merge_data(file_name_str, folder, output_path)

자동 계산된 time_gap: 0.000500
병합된 파일이 저장되었습니다: c:\Users\limye\OneDrive - gachon.ac.kr\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\2. Collecting data\250624_12개의 샘플\minimized_data\shape\00Merge.csv
